In [44]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict,Annotated
from pydantic import BaseModel, Field
import operator

In [45]:
load_dotenv()

True

In [46]:
model = ChatOpenAI(model ="gpt-4o-mini")

In [47]:
class EvaluationSchema(BaseModel):

    feedback:str = Field(description="Detailed feedback for the essay")
    score:int = Field(description="Score out of 10",ge=0, le=10)

In [48]:
structure_model = model.with_structured_output(EvaluationSchema)

In [49]:
essay = """India has rapidly emerged as a global hub for artificial intelligence (AI), leveraging its vast talent pool, thriving digital public infrastructure, and proactive government initiatives to shape the future of AI both domestically and internationally. The role of India in AI is multifaceted, influencing economic development, social transformation, and global research trends.

**1. Government Vision and Initiatives**

The Indian government has strategically positioned AI as a cornerstone of national growth. Through missions like the IndiaAI Mission—recently allocated over ₹10,300 crore—the country is investing in scaling AI infrastructure, democratizing high-performance computing with over 10,000 GPUs, and fostering innovation through public-private partnerships[7][4]. The goal is to create an inclusive, responsible, and robust AI ecosystem, enabling startups, researchers, and industry to build solutions tailored to India's unique needs[7][4].

Flagship programs such as Digital India, the National Strategy for Artificial Intelligence, and Centres of Excellence reflect a holistic vision: integrating AI into governance, healthcare, agriculture, and education, strengthening indigenous capabilities, and establishing India as a global leader[8][4][6].

**2. Research, Startups, and Industry Collaboration**

India's vibrant startup ecosystem fuels its AI revolution. Indian startups and tech giants are pioneering solutions in sectors ranging from banking and agriculture to healthcare, driving productivity and innovation[1][5]. The country ranks fifth worldwide for generative AI patents and is expected to contribute around $500 billion from AI to its GDP[5]. Collaborations with global technology leaders, like Google and Microsoft, bolster India's AI landscape and attract significant investments due to its cost-effective talent pool[5].

Academic and research institutions such as the Indian Institute of Science and Indian Statistical Institute publish breakthrough research, supported by government-industry partnerships[3][4]. Homegrown models and projects, like BharatGen (India's multilingual, multimodal AI model), aim to reduce dependence on foreign platforms, ensure security for critical sectors, and promote sustainable innovation[3][4].

**3. Digital Public Infrastructure and Data**

India’s unique digital public infrastructure (DPI)—including Aadhaar, UPI, and DigiLocker—has redefined digital inclusion by offering open, scalable platforms on which AI solutions can be built. The IndiaAI Dataset Platform provides seamless access to high-quality, non-personal datasets, empowering the development of applications in financial services, governance, and social sectors[4].

This combination of government investment, foundational datasets, and public infrastructure allows Indian AI applications to address local language, cultural, and socio-economic diversity—a distinguishing feature compared to other leading AI countries[3][4].

**4. Talent and Education**

India boasts the world’s largest AI workforce, projected to reach 1.25 million by 2027, outpacing many developed countries[5]. Affordable, high-quality education in science and engineering, ongoing upskilling initiatives, and the drive to integrate AI into curricula ensure a continuous supply of skilled professionals[5].

**5. Global Position and Partnerships**

While the US and China remain ahead in AI R&D, India is closing the gap through pragmatic policy, global partnerships, and leadership in “AI for Good”—solutions targeting healthcare, climate, education, and social development, both within India and across developing economies[3][6]. India ranks tenth globally for private AI investment and is an attractive market for AI platforms like ChatGPT, with one of the world’s largest user bases[3].

Through the Indo-US Science and Technology Forum and initiatives like the Defense Artificial Intelligence Dialogue, India has deepened technological cooperation in AI research and cyber training. These partnerships enable knowledge exchange and accelerate India’s position as a solution provider for emerging global challenges[3].

**6. Ethical AI, Regulation, and Challenges**

India’s regulatory approach emphasizes “techno-legal” solutions—balancing innovation with accountability by investing in safeguards against deep fakes, privacy risks, and cyber threats rather than imposing restrictive regulations[4]. However, challenges persist: data privacy, skill shortages, resource allocation, and the need for indigenous innovation remain areas for further development[4][6].

**7. Future Prospects**

Looking ahead, India aspires to become a global AI powerhouse by 2047, with AI integrated across every pillar of the economy and society[4]. The emphasis on indigenous models, digital inclusion, and global collaboration positions India to not only benefit from AI, but also lead its ethical and sustainable development worldwide[4][5][3].

In summary, India plays a transformational role in AI by building accessible, inclusive infrastructure; fostering innovation; developing a skilled workforce; collaborating globally; and championing responsible technology. This multifaceted strategy is set to make India a central force in shaping the future of AI for economic growth, social good, and global leadership[4][5][3].

"""

In [50]:
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n{essay}'
structure_model.invoke(prompt).score

8

In [51]:
class UPSCState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_score:Annotated[list[int],operator.add]
    avg_score:float

In [ ]:
def evaluate_language(state: UPSCState):

    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n{state["essay"]}'
    output = structure_model.invoke(prompt)
    
    return {'language_feedback': output.feedback, 'individual_score':  [output.score]}

In [53]:
def evaluate_analysis(state: UPSCState):

    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n{state["essay"]}'
    output = structure_model.invoke(prompt)
    
    return {'analysis_feedback': output.feedback, 'individual_score': [output.score]}

In [54]:
def evaluate_thought(state: UPSCState):

    prompt = f'Evaluate the clarity of the thought of the following essay and provide a feedback and assign a score out of 10 \n{state["essay"]}'
    output = structure_model.invoke(prompt)
    
    return {'evaluate_thought': output.feedback, 'individual_score': [output.score]}

In [55]:
def final_evaluation(state: UPSCState):

    # summery feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language_feedback: {state["language_feedback"]}\n depth of analysis_feedback: {state["analysis_feedback"]}\n clarity of thought feedback: {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content
    
    #
    avg_score = sum(state['individual_score'])/ len(state['individual_score'])
    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}

In [56]:
graph = StateGraph(UPSCState)

graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)

#edges
graph.add_edge(START, 'evaluate_language')
# graph.add_edge(START, 'evaluate_analysis')
# graph.add_edge(START, 'final_evaluation')
graph.add_edge('evaluate_language', 'evaluate_analysis')    
graph.add_edge('evaluate_analysis', 'evaluate_thought')
graph.add_edge('evaluate_thought', 'final_evaluation')

graph.add_edge('final_evaluation', END)

workflow = graph.compile()

In [57]:
initial_state = {
    'essay': essay,
    'language_feedback': "",
    'analysis_feedback': "",
    'clarity_feedback': "",
    'overall_feedback': "",
    'individual_score': [],
    'avg_score': 0.0,
}

In [58]:
result = workflow.invoke(initial_state)
print(result)

{'essay': "India has rapidly emerged as a global hub for artificial intelligence (AI), leveraging its vast talent pool, thriving digital public infrastructure, and proactive government initiatives to shape the future of AI both domestically and internationally. The role of India in AI is multifaceted, influencing economic development, social transformation, and global research trends.\n\n**1. Government Vision and Initiatives**\n\nThe Indian government has strategically positioned AI as a cornerstone of national growth. Through missions like the IndiaAI Mission—recently allocated over ₹10,300 crore—the country is investing in scaling AI infrastructure, democratizing high-performance computing with over 10,000 GPUs, and fostering innovation through public-private partnerships[7][4]. The goal is to create an inclusive, responsible, and robust AI ecosystem, enabling startups, researchers, and industry to build solutions tailored to India's unique needs[7][4].\n\nFlagship programs such as 